# 1) read Dataset


In [3]:
import importlib

helper_function = importlib.import_module("HelperFucntions")
importlib.reload(helper_function)
from HelperFucntions import *
importlib.reload(helper_function)

preprocessor_module = importlib.import_module("Preprocessor")
importlib.reload(preprocessor_module)
from Preprocessor import *
importlib.reload(preprocessor_module)

# create sequences for variable window sizes
def create_sequences(features, labels, window_size, step_size=1):
    x, y = [], []
    for i in range(0, len(features) - window_size + 1, step_size):
        x.append(features[i:(i + window_size)])
        y.append(labels[i + window_size - 1])
    return np.array(x), np.array(y)



import torch
import pandas as pd
from os import path


Report.green(f'Reading dataset for is_binary=True and is_supervised={False} started ...!')
data = pd.read_csv(path.join('..','..','input','Dataset.csv'))
x_train, x_val, x_test, y_train, y_val, y_test = preprocess_data(data= data, is_binary=True, is_supervised=False)

X_train, X_val, y_val, X_test, y_test =  torch.tensor(x_train, dtype=torch.float32),  torch.tensor(x_val, dtype=torch.float32),  torch.tensor(y_val, dtype=torch.int32), torch.tensor(x_test, dtype=torch.float32),  torch.tensor(y_test, dtype=torch.int32)


if torch.isnan(X_train).any():
    print("❗ Input tensor contains NaN values.")
else:
    print("✅ No NaNs in input tensor.")




[16:24:04] Reading dataset for is_binary=True and is_supervised=False started ...!
Label Distribution for train:
0: 1.000%
Label Distribution for validation:
0: 0.658%
1: 0.342%
Label Distribution for test:
0: 0.732%
1: 0.268%
data shape: (45718, 52)
x_train shape: (17372, 52)
y_val shape: (19201,)
y_test shape: (9145,)
✅ No NaNs in input tensor.


# 2) Grid Search


In [2]:
# def create_sequences(features, labels, window_size, step_size=1):
#     x, y = [], []
#     for i in range(0, features.size(0) - window_size + 1, step_size):
#         x.append(features[i:i + window_size])
#         y.append(labels[i + window_size - 1])
#     return torch.stack(x), torch.tensor(y)
#
#
# window_size = 20
# X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
# X_val_seq, y_val_seq = create_sequences(X_val, y_val, window_size)
# X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

In [6]:
models_module = importlib.import_module("Models")
importlib.reload(models_module)
from Models import *
importlib.reload(models_module)
#
#
gridsearch_module = importlib.import_module("GridSearch")
importlib.reload(gridsearch_module)
from GridSearch import *
importlib.reload(gridsearch_module)


import json

np.random.seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Options: AutoEncoder, VariationalAutoEncoder, ConditionalVariationalAutoEncoder, LSTMAutoEncoder
ModelClass = LSTMAutoEncoder  # 👈 change this to the model you want

# ----- Step 3: Define parameter grid -----
if ModelClass.__name__ in ["AutoEncoder", "VariationalAutoEncoder", "ProbabilisticVariationalEncoder"]:
    # param_grid = {
    #     "hidden_layers": [[64,32]],
    #     "latent_dim": [10, 20, 30],
    #     "dropout": [0.0],
    #     "activation": ["relu"],
    #     "batch_size": [32],
    #     "lr": [ 1e-3],
    #     "optimizer": ["adam"],
    #     "epochs": [ 2, 5]
    # }
    param_grid = {
        "hidden_layers": [[64, 32], [128, 64], [64], [32]],
        "latent_dim": [10, 20, 30],
        "dropout": [0.0, 0.1],
        "activation": ["relu", "leaky_relu"],
        "batch_size": [32],
        "lr": [ 1e-3],
        "optimizer": ["adam", "sgd"],
        "epochs": [10]
    }
    input_data = X_train
    cond_data = None
    input_dim = X_train.shape[1]
    test_data_tensor = X_val
    test_labels = y_val
    test_cond_tensor=None




elif ModelClass.__name__ == "ConditionalVariationalAutoEncoder":
    X_tensor = X_train[:, :47]  # normal features
    C_tensor = X_train[:, 47:]  # one-hot protocol features
    param_grid = {
        "hidden_layers": [[64, 32], [128, 64], [64], [32]],
        "latent_dim": [10, 20 , 30],
        "dropout": [0.0, 0.1],
        "activation": ["relu", "leaky_relu"],
        "batch_size": [32],
        "lr": [ 1e-3],
        "optimizer": ["adam", "sgd"],
        "epochs": [50]
    }
    input_data = X_tensor
    cond_data = C_tensor
    input_dim = X_tensor.shape[1]
    test_data_tensor = X_val[:, :47]
    test_labels = y_val
    test_cond_tensor=X_val[:, 47:]



elif ModelClass.__name__ == "LSTMAutoEncoder":
    param_grid = {
        "hidden_dim": [128, 64, 32],
        "latent_dim": [10, 20, 30],
        "num_layers": [1, 2, 3],
        "batch_size": [32],
        "lr": [1e-3],
        "optimizer": ["adam", "sgd"],
        "epochs": [50],
        "window_size": [5, 10, 20],

    }

    # param_grid = {
    #     "hidden_dim": [32],
    #     "latent_dim": [10],
    #     "num_layers": [1, ],
    #     "batch_size": [32],
    #     "lr": [1e-3],
    #     "optimizer": ["adam", "sgd"],
    #     "epochs": [100],
    #     "window_size": [20],
    #
    # }
    input_data = X_train
    cond_data = None
    input_dim = X_train.shape[1]
    test_data_tensor = X_val
    test_labels = y_val
    test_cond_tensor=None
    # input_data = X_train_seq
    # cond_data = None
    # input_dim = X_train_seq.shape[2]
    # test_data_tensor = X_val_seq
    # test_labels = y_val_seq
    # test_cond_tensor=None

else:
    raise ValueError("Unsupported model type")

# ----- Step 4: Run grid search -----
print("\nRunning: ", ModelClass.__name__)
best_config, best_config_by_accuracy, all_results = grid_search_autoencoder(
    data_tensor=input_data,
    param_grid=param_grid,
    input_dim=input_dim,
    model_class=ModelClass,
    device=device,
    cond_tensor=cond_data,
    verbose=True,
    test_data_tensor=test_data_tensor,
    test_labels=test_labels,
    test_cond_tensor=test_cond_tensor,  # optional
    result_path = f"GridSearchResults/cach_{ModelClass.__name__}"
)
print("\n✅ Best configuration:", best_config)
print("\n🎯 Best Config by Test Accuracy:", best_config_by_accuracy)

# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = ModelClass.__name__
outfile = f"GridSearchResults/results_{model_name}_{timestamp}.json"

safe_data = convert_to_serializable({
    "model": model_name,
    "search_space":param_grid,
    "best_config": best_config,
    "best_config_by_accuracy": best_config_by_accuracy,
    "all_results": all_results
})

with open(outfile, "w") as f:
    json.dump(safe_data, f, indent=2)

print(f"\n📁 Results saved to {outfile}")




Running:  LSTMAutoEncoder

⚠️ Skipping already evaluated config 1: {'hidden_dim': 128, 'latent_dim': 10, 'num_layers': 1, 'batch_size': 32, 'lr': 0.001, 'optimizer': 'adam', 'epochs': 50, 'window_size': 5}

⚠️ Skipping already evaluated config 2: {'hidden_dim': 128, 'latent_dim': 10, 'num_layers': 1, 'batch_size': 32, 'lr': 0.001, 'optimizer': 'adam', 'epochs': 50, 'window_size': 10}

⚠️ Skipping already evaluated config 3: {'hidden_dim': 128, 'latent_dim': 10, 'num_layers': 1, 'batch_size': 32, 'lr': 0.001, 'optimizer': 'adam', 'epochs': 50, 'window_size': 20}

⚠️ Skipping already evaluated config 4: {'hidden_dim': 128, 'latent_dim': 10, 'num_layers': 1, 'batch_size': 32, 'lr': 0.001, 'optimizer': 'sgd', 'epochs': 50, 'window_size': 5}

⚠️ Skipping already evaluated config 5: {'hidden_dim': 128, 'latent_dim': 10, 'num_layers': 1, 'batch_size': 32, 'lr': 0.001, 'optimizer': 'sgd', 'epochs': 50, 'window_size': 10}

⚠️ Skipping already evaluated config 6: {'hidden_dim': 128, 'latent_dim

# Final Data

In [10]:
# 4. Train final model on full data
final_dataset = TensorDataset(X_train)
final_model = AutoEncoder(
    input_dim=52,
    encoder_layers=best_config["encoder_layers"],
    decoder_layers=best_config["decoder_layers"],
    dropout=best_config["dropout"],
    activation=best_config["activation"]
)

train_autoencoder(
    final_model,
    train_data=final_dataset,
    batch_size=best_config["batch_size"],
    lr=best_config["lr"],
    optimizer_name=best_config["optimizer"],
    num_epochs=best_config["epochs"],
    device="cpu"
)

# 5. Compute anomaly scores
reconstruction_errors = compute_reconstruction_error(final_model, data_tensor)
threshold = np.percentile(reconstruction_errors, 95)
anomalies = reconstruction_errors > threshold

print("\n📊 Total anomalies detected:", np.sum(anomalies))